In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Estudio de desempleo

## Paso 1: importado de datos CSV

In [6]:
#IMPORTAR CSV
df=pd.read_csv("Data/desempleo.csv")
print('Forma del Dataframe: ',df.shape)
print('Tamaño del Dataframe: ',df.size)

Forma del Dataframe:  (203520, 35)
Tamaño del Dataframe:  7123200


In [7]:
#ESCRIBIR CABECERA (5 ROWS)
df.head(5)

,STRUCTURE,STRUCTURE_ID,ACTION,FREQ_ID,FREQ_NAME,REF_AREA_ID,REF_AREA_NAME,INDICATOR_ID,INDICATOR_NAME,SEX_ID,...,DATABASE_ID_ID,DATABASE_ID_NAME,UNIT_MULT_ID,UNIT_MULT_NAME,UNIT_TYPE_ID,UNIT_TYPE_NAME,OBS_STATUS_ID,OBS_STATUS_NAME,OBS_CONF_ID,OBS_CONF_NAME
0,datastructure,WB.DATA360:DS_DATA360(1.1),I,A,Annual,ABW,Aruba,WB_GS_SL_UEM_ZS,Unemployment (%),F,...,WB_GS,Gender Statistics,0,Units,RATIO,Ratio,O,Missing value,PU,Public
1,datastructure,WB.DATA360:DS_DATA360(1.1),I,A,Annual,AFE,Africa Eastern and Southern,WB_GS_SL_UEM_ZS,Unemployment (%),F,...,WB_GS,Gender Statistics,0,Units,RATIO,Ratio,O,Missing value,PU,Public
2,datastructure,WB.DATA360:DS_DATA360(1.1),I,A,Annual,AFG,Afghanistan,WB_GS_SL_UEM_ZS,Unemployment (%),F,...,WB_GS,Gender Statistics,0,Units,RATIO,Ratio,O,Missing value,PU,Public
3,datastructure,WB.DATA360:DS_DATA360(1.1),I,A,Annual,AFW,Africa Western and Central,WB_GS_SL_UEM_ZS,Unemployment (%),F,...,WB_GS,Gender Statistics,0,Units,RATIO,Ratio,O,Missing value,PU,Public
4,datastructure,WB.DATA360:DS_DATA360(1.1),I,A,Annual,AGO,Angola,WB_GS_SL_UEM_ZS,Unemployment (%),F,...,WB_GS,Gender Statistics,0,Units,RATIO,Ratio,O,Missing value,PU,Public


In [8]:
#EXTRAER EL ÚLTIMO AÑO
ultimoAnio=df["time"].max()
print("año:",ultimoAnio)
df_LastYear=df[df["time"]==ultimoAnio]
print('Forma del Dataframe: ',df_LastYear.shape)
print('Tamaño del Dataframe: ',df_LastYear.size)

KeyError: 'time'

In [9]:
df_LastYear.head()

,unit,n_portion,sex,age,country,time,value
0,PC,0,F,TOTAL,AT,2019,29.1
1,PC,0,F,TOTAL,BE,2019,17.1
2,PC,0,F,TOTAL,BG,2019,54.2
3,PC,0,F,TOTAL,CY,2019,32.7
4,PC,0,F,TOTAL,CZ,2019,49.6


## Paso 2: Limpieza de datos

In [ ]:
#DESCRIBO EL DATAFRAME PARA LIMPIAR CORRECTAMENTE POR COLUMNAS
df_LastYear.info()
print('columnas del DataFrame:',df_LastYear.columns)

#LIMPIEZA PREVIA DE TIPOS
df["country"] = df["country"].astype(str).str.strip()
df["sex"] = df["sex"].astype(str).str.strip()
df["time"] = df["time"].astype(str).str.strip()
df["value"] = pd.to_numeric(df["value"], errors="coerce")

### Limpieza profunda de datos: rellenar nulos, eliminar datos innecesarios o que distorsionen...etc

In [ ]:
#VERIFICO QUE NO HAY PAISES EXTRAÑOS O GRUPOS DE PAISES
print('Valores de country en DataFrame original:',df["country"].unique())

# Eliminar agregaciones de la UE
paises_a_eliminar = ["EU28", "EU27_2020"]
df_limpio = df_LastYear[~df_LastYear["country"].isin(paises_a_eliminar)]

print('Países en DataFrame limpio:',df_limpio["country"].unique())
print("_"*50)

#VERIFICO LOS VALORES DE SEXO
print('Valores de sexo en DataFrame original:',df["sex"].unique())
df_limpio = df_limpio[df_limpio["sex"].isin(["F", "M"])]
print('Valores de sexo en DataFrame limpio:',df_limpio["sex"].unique())
print("_"*50)

#COMPROBAMOS QUE LOS RANGOS DE EDAD SON CORRECTOS
print('Valores de age en DataFrame original:',df["age"].unique())

import re
pattern = r"^Y\d{1,3}-\d{1,3}$"   
mask_edades = df_limpio["age"].str.match(pattern, na=False)

#FILTRAR Y VER
df_limpio = df_limpio[mask_edades].copy()
print("Edades únicas después:",(df_limpio["age"].unique()))

elegidos = ["Y15-24","Y25-34","Y35-44","Y45-54","Y45-64"]
df_limpio = df_limpio[df_limpio["age"].isin(elegidos)].copy()
print('Rangos elegidos de edad:',df_limpio["age"].unique())


print("_"*50)

#EL PRIMER PROBLEMA QUE OBSERVO, ES QUE HAY "value" VACIOS, relleno con la media.
#luego veo que hay ceros, que también los voy a reemplazar por la media.
#Para mayor precisión, lo hago con la media global, no con la de año.

#ANALIZO VALORES NULOS Y CEROS Y REEMPLAZO LOS ZEROS POR NAN PARA NO INFLUIR EN LA MEDIA
print('Cantidad de valores nulos en el dataframe original:',df_LastYear["value"].isna().replace(0,np.nan).sum())
print('Cantidad de ceros en el dataframe original:',(df_LastYear["value"]==0).sum())
print("_"*50)

#CALCULO LA MEDIA GLOBAL
media_valor=df_limpio["value"].mean().round(1)
print('La media global de valor de consumo es:',media_valor)
print("_"*50)

#REEMPLAZO VALORES NULOS Y CEROS POR LA MEDIA GLOBAL
#df_limpio = df_LastYear.copy()
df_limpio["value"] = df_limpio["value"].fillna(media_valor)
# Reemplazar ceros por la media por país

def calcular_media_sin_ceros(grupo):
    """Calcula la media excluyendo los ceros"""
    return grupo[grupo != 0].mean()

media_por_pais = df_limpio.groupby("country")["value"].transform(calcular_media_sin_ceros)
df_limpio.loc[df_limpio["value"]==0, "value"] = media_por_pais

print('Cantidad de valores nulos actualizada:',df_limpio["value"].isna().sum())
print('Cantidad de ceros actualizada:',(df_limpio["value"]==0).sum())
print("_"*50)





## Paso 3: análisis

### Muestra de promedio de consumo en gramos por país

In [ ]:
#UNA VEZ LIMPIO, PRETENDO MOSTRAR UN RESUMEN DE MEDIA POR PAIS

df_resumen=df_limpio.groupby("country")["value"].mean().reset_index()
df_resumen.set_index("country", inplace=True)
df_resumen.sort_values(by="value", ascending=False, inplace=True)   
print('Resumen de los primeros cinco paises con mayor consumo diario de frutas y verduras:')
print(df_resumen.head())
print("_"*50)
print('Resumen de los últimos cinco paises con menor consumo diario de frutas y verduras:')
print(df_resumen.tail())

## Paso 4: comunicación de datos y visualización

### Mostramos gráficos de consumo por país

In [ ]:
#MUESTRO UN GRÁFICO PARA VER MEJOR LA DIFERENCIA ENTRE PAÍSES

paises = df_resumen.index
consumo = df_resumen["value"]

plt.figure(figsize=(12, 6))
plt.bar(paises, consumo, color='skyblue')
plt.xlabel('Países')
plt.ylabel('Consumo Promedio de Frutas y Verduras (g/día)')
plt.title('Consumo Promedio de Frutas y Verduras por País en Europa')
plt.tight_layout()
plt.show()

In [ ]:
#OTRO GRÁFICO DE BARRAS CON LÍMITES MÁS JUSTOS
plt.figure(figsize=(12, 6))
plt.bar(paises, consumo)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.ylim(np.min(consumo), np.max(consumo)+0.2)  # Ajusta los límites del eje y para enfocarte en el rango relevante
plt.xlabel('Países')    
plt.ylabel('Consumo Promedio de Frutas y Verduras (g/día)')
plt.title('Consumo Promedio de Frutas y Verduras por País en Europa (Enfoque)')
plt.tight_layout()      
plt.show()

### Analizamos el sexo en relación al consumo y mostramos gráfico

In [ ]:
#AHORA QUIERO VER LA DIFERENCIA ENTRE SEXOS EN CADA PAÍS
#AGRUPO POR SEXOS Y DESPUÉS HAGO UN PIVOT PARA TENERLOS EN COLUMNAS
df_sexo = df_limpio.groupby(["country","sex"])["value"].mean().reset_index()
df_sexo_pivot = df_sexo.pivot(index="country", columns="sex", values="value")
df_sexo_pivot["diferencia"] = df_sexo_pivot.get("F", pd.Series(dtype=float)) - df_sexo_pivot.get("M", pd.Series(dtype=float))
df_sexo_pivot.sort_values(by="diferencia", ascending=False, inplace=True)

#EXTRAIGO ARRAYS PARA EL GRÁFICO
f = df_sexo_pivot["F"]
m = df_sexo_pivot["M"]

paises = df_sexo_pivot.index
x = np.arange(len(paises))

plt.figure(figsize=(12, 6))
plt.plot(x, f, label='Mujeres (F)', marker='o', color='pink')
plt.plot(x, m, label='Hombres (M)', marker='o')
plt.xticks(x, paises)
plt.title('Consumo Promedio de Frutas y Verduras por País y Sexo en Europa')
plt.xlabel('Países')
plt.ylabel('Consumo Promedio de Frutas y Verduras (g/día)')
plt.legend()

plt.tight_layout()
plt.show()


### Análisis de consumo por rangos de edades

In [ ]:
#VAMOS A HACER UN ANÁLISIS POR EDADES TAMBIÉN
df_edad = df_limpio.groupby(["age"])["value"].mean()
print(df_edad.head())


In [ ]:
#GRAFICO DE BARRAS PARA ANALIZAR POR RANGOS DE EDADES
rangos_edad=df_edad.index
consumo_edad=df_edad.values

# Incrementos de 0.05 para máxima granularidad
yticks = np.arange(min(consumo_edad), max(consumo_edad) + 0.05, 0.05)

plt.figure(figsize=(10, 5))
#limites más justos para ver mejor la diferencia
plt.ylim(33.3, 33.65)
plt.bar(rangos_edad, consumo_edad, color='orange')
plt.xlabel('Rangos de Edad')
plt.ylabel('Consumo Promedio de Frutas y Verduras (g/día)') 
plt.title('Consumo Promedio de Frutas y Verduras por Rango de Edad en Europa')
plt.yticks(yticks)
plt.tight_layout()
plt.show()

## Paso extra: preparación de datos para uso de compañeros

### Aquí escribo un resumen de los dataframes y series utilizados en limpio para facilitar la extracción de los mismos por parte de mis compañeros

In [ ]:
#DATAFRAMES EXTRAIDOS
df_final=df_limpio.copy()
df_paises = df_resumen.copy()
df_sexos = df_sexo_pivot.copy()
df_edades = df_edad.copy()
#_______________________________________________________________________________________
print('RESUMEN DE DATAFRAMES EXTRAIDOS:')
print("-"*50)

print("DATAFRAME LIMPIO:")
print(df_final.head())
print("_"*50)

print("VALORES EN GRAMOS DE CONSUMO PROMEDIO POR PAÍS:")
print(df_paises.head())
print("_"*50)

print("VALORES EN GRAMOS DE CONSUMO PROMEDIO POR SEXO Y PAÍS:")
print(df_sexos.head())
print("_"*50)

print("VALORES EN GRAMOS DE CONSUMO PROMEDIO POR RANGO DE EDAD:")
print(df_edades.head())
print("_"*50)

In [ ]:
#SERIES EXTRAIDAS
serie_paises = df_resumen["value"].copy() 
serie_hombres = df_sexo_pivot["M"].copy()
serie_mujeres = df_sexo_pivot["F"].copy() 
serie_consumo_edad = df_edad.values.copy()